In [1]:
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

import edward as ed
import matplotlib.pyplot as plt
import numpy as np
import tensorflow as tf
import math

from os import listdir
from os.path import isfile, join
import os.path
import random
import scipy.special as sp

from edward.models import Poisson,Gamma

plt.style.use('ggplot')

In [2]:
X = np.loadtxt('./pre_processed_data/bibtex/X_train.txt',delimiter=',')
train_mask = np.loadtxt('./pre_processed_data/bibtex/x_train_mask.txt')
y = X*train_mask
tot = 101
mae_val = np.zeros(11)

In [3]:
def mae(t,result):
    error = 0.0
    count = 0.0
    for i in range(0,users):
        for j in range(0,items):
            if train_mask[i,j] == 0:
                error += abs(X[i,j]-result[i,j])
                count += 1
    error /= count
    #error = math.sqrt(error)
    mae_val[t] = error

In [4]:
def check(t):
    q_theta = Gamma(gam_shp,gam_rte)
    q_beta = Gamma(lam_shp,lam_rte)
    sess = tf.InteractiveSession()
    init = tf.global_variables_initializer()
    init.run()
    no_sample = 100
    beta_sample = q_beta.sample(no_sample).eval()
    theta_sample = q_theta.sample(no_sample).eval()
    result = np.zeros([users,items])
    for i in range(0,no_sample):
        result = np.add(result,np.matmul(theta_sample[i],np.transpose(beta_sample[i])))
    result /= no_sample
    mae(t,result)

In [5]:
K = 10
users = X.shape[0]
items = X.shape[1]
a = a_c = c = c_c = 0.3
b_c = d_c = 1.0      

In [6]:
kappa_shp =  np.random.uniform(low=0.1,size=users)
kappa_rte = np.random.uniform(low=0.1,size=users)
tau_shp = np.random.uniform(low=0.1,size=items)
tau_rte = np.random.uniform(low=0.1,size=items)
phi = np.zeros([users,items,K])
gam_shp = np.random.uniform(low=0.1,size=[users,K])
gam_rte = np.random.uniform(low=0.1,size=[users,K])
lam_shp = np.random.uniform(low=0.1,size=[items,K])
lam_rte = np.random.uniform(low=0.1,size=[items,K])

In [7]:
for u in range(0,users):
    kappa_shp[u] = a_c + K*a
for i in range(0,items):
    tau_shp[i] = c_c + K*c
for ite in range(0,tot):
    if ite%5==0:
        print(ite)
    
    for u in range(0,users):
        for i in range(0,items):
            if y[u,i] > 0:
                phi[u,i,:]= np.exp(sp.digamma(gam_shp[u,:])-np.log(gam_rte[u,:])+sp.digamma(lam_shp[i,:])-np.log(lam_rte[i,:]))
                norm = np.sum(phi[u,i,:])
                phi[u,i,:] = phi[u,i,:]/norm
    
    for u in range(0,users):
        for k in range(0,K):
            gam_shp[u,k] = a + np.inner(y[u,:],phi[u,:,k])
            gam_rte[u,k] = (kappa_shp[u]/kappa_rte[u]) + np.sum(lam_shp[:,k]/lam_rte[:,k])
        kappa_rte[u] = (a_c/b_c) + np.sum(gam_shp[u,:]/gam_rte[u,:])
    
    for i in range(0,items):
        for k in range(0,K):
            lam_shp[i,k] = c + np.inner(y[:,i],phi[:,i,k])
            lam_rte[i,k] = (tau_shp[i]/tau_rte[i]) + np.sum(gam_shp[:,k]/gam_rte[:,k])
        tau_rte[i] = (c_c/d_c) + np.sum(lam_shp[i,:]/lam_rte[i,:])
    
    if ite%10 == 0:
        check(int(ite/10))

0
5


KeyboardInterrupt: 